# Pydantic

## Introducción

[Pydantic](https://pydantic-docs.helpmanual.io/) es una librería de python centrada en tres principios:

* Validación de datos y gestión de la configuración mediante anotaciones de tipo Python.
* Aplicar sugerencias de tipo en tiempo de ejecución y proporciona errores fáciles de usar cuando los datos no son válidos.
* Definir cómo deben estar los datos en Python canónico puro.

Veamos un ejemplo:

In [1]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel

In [2]:
class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []


external_data = {
    'id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
}

In [3]:
user = User(**external_data)
print(user.id)

123


In [4]:
print(repr(user.signup_ts))

datetime.datetime(2019, 6, 1, 12, 22)


In [5]:
print(user.friends)

[1, 2, 3]


In [6]:
print(user.dict())

{'id': 123, 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'friends': [1, 2, 3], 'name': 'John Doe'}


Que está pasando aqui:

* `id` es de tipo int; la declaración de solo anotación le dice a Pydantic que este campo es obligatorio. Las cadenas, bytes o flotantes se convertirán en ints si es posible; de lo contrario, se generará una excepción.

* `name` se infiere como una cadena del valor predeterminado proporcionado; debido a que tiene un valor predeterminado, no es necesario.

* `signup_ts` es un campo de fecha y hora que no es obligatorio (y toma el valor None si no se proporciona). pydantic procesará una marca de tiempo de Unix int (por ejemplo, 1496498400) o una cadena que represente la fecha y la hora.

* `friends` usa el sistema de escritura de Python y requiere una lista de entradas. Al igual que con `id`, los objetos de tipo entero se convertirán en enteros.

Si la validación falla, pydantic generará un error con un desglose de lo que estaba mal:

In [7]:
from pydantic import ValidationError

try:
    User(signup_ts='broken', friends=[1, 2, 'not number'])
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "id"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  },
  {
    "loc": [
      "signup_ts"
    ],
    "msg": "invalid datetime format",
    "type": "value_error.datetime"
  },
  {
    "loc": [
      "friends",
      2
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  }
]


## Models

El medio principal para definir objetos en Pydantic es a través de modelos (los modelos son simplemente clases que heredan de `BaseModel)`.

Puede pensar en los modelos como similares a los tipos en lenguajes estrictamente typed, o como los requisitos de un solo punto final en una API.

Los datos que no son de confianza se pueden pasar a un modelo y, después de analizar y validar, Pydantic garantiza que los campos de la instancia del modelo resultante se ajustarán a los tipos de campo definidos en el modelo.


In [8]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name = 'Jane Doe'

### Propiedades del modelo
El ejemplo anterior solo muestra la punta del iceberg de lo que pueden hacer los modelos. Los modelos poseen los siguientes métodos y atributos:

* `dict()`
returns a dictionary of the model's fields and values; cf. exporting models
* `json()`
returns a JSON string representation dict(); cf. exporting models
* `copy()`
returns a deep copy of the model; cf. exporting models
* `parse_obj()`
a utility for loading any object into a model with error handling if the object is not a dictionary; cf. helper functions
* `parse_raw()`
a utility for loading strings of numerous formats; cf. helper functions
* `parse_file()`
like parse_raw() but for files; cf. helper function
* `from_orm()`
loads data into a model from an arbitrary class; cf. ORM mode
* `schema()`
returns a dictionary representing the model as JSON Schema; cf. Schema
* `schema_json()`
returns a JSON string representation of schema(); cf. Schema
* `construct()`
a class method for creating models without running validation; cf. Creating models without validation
* `__fields_set__`
Set of names of fields which were set when the model instance was initialised
* `__fields__`
a dictionary of the model's fields
* `__config__`
the configuration class for the model, cf. model config

## Validators
La validación personalizada y las relaciones complejas entre objetos se pueden lograr utilizando el decorador `validator`.

In [9]:
from pydantic import BaseModel, ValidationError, validator


class UserModel(BaseModel):
    name: str
    username: str
    password1: str
    password2: str

    @validator('name')
    def name_must_contain_space(cls, v):
        if ' ' not in v:
            raise ValueError('must contain a space')
        return v.title()

    @validator('password2')
    def passwords_match(cls, v, values, **kwargs):
        if 'password1' in values and v != values['password1']:
            raise ValueError('passwords do not match')
        return v

    @validator('username')
    def username_alphanumeric(cls, v):
        assert v.isalnum(), 'must be alphanumeric'
        return v

In [10]:
# correcto
user = UserModel(
    name='samuel colvin',
    username='scolvin',
    password1='zxcvbn',
    password2='zxcvbn',
)
print(user)

name='Samuel Colvin' username='scolvin' password1='zxcvbn' password2='zxcvbn'


In [11]:
# incorrecto

try:
    UserModel(
        name='samuel',
        username='scolvin',
        password1='zxcvbn',
        password2='zxcvbn2',
    )
except ValidationError as e:
    print(e)

2 validation errors for UserModel
name
  must contain a space (type=value_error)
password2
  passwords do not match (type=value_error)


Algunas cosas a tener en cuenta sobre los validadores:

* los validadores son "métodos de clase", por lo que el primer valor de argumento que reciben es la clase `UserModel`, no una instancia de `UserModel`.
* el segundo argumento es siempre el valor del campo a validar; se puede nombrar como quieras
* también puede agregar cualquier subconjunto de los siguientes argumentos a la firma (los nombres deben coincidir):
 * `values`: un dictado que contiene el mapeo de nombre a valor de cualquier campo validado previamente
 * `config`: la configuración del modelo
 * `field`: el campo que se está validando
 * `**kwargs`: si se proporciona, esto incluirá los argumentos anteriores que no se enumeran explícitamente en la firma
 
* los validadores deben devolver el valor analizado o generar un `ValueError`, `TypeError` o `AssertionError` (se pueden usar declaraciones de aserción).

* donde los validadores se basan en otros valores, debe tener en cuenta que:

 * La validación se realiza en el orden en que se definen los campos. En el ejemplo anterior, `password2` tiene acceso a `password1` (y `name`), pero `password1` no tiene acceso a `password2`. 

 * Si la validación falla en otro campo (o ese campo falta), no se incluirá en los valores, por lo tanto, `if 'password1' in values and ...` en este ejemplo.

## Model Config

El comportamiento de Pydantic se puede controlar a través de la clase `Config` en un modelo.

* `title`
the title for the generated JSON Schema
* `anystr_strip_whitespace`
whether to strip leading and trailing whitespace for str & byte types (default: False)
* `min_anystr_length`
the min length for str & byte types (default: 0)
* `max_anystr_length`
the max length for str & byte types (default: 2 ** 16)
* `validate_all`
whether to validate field defaults (default: False)
* `extra`
whether to ignore, allow, or forbid extra attributes during model initialization. Accepts the string values of 'ignore', 'allow', or 'forbid', or values of the Extra enum (default: Extra.ignore). 'forbid' will cause validation to fail if extra attributes are included, 'ignore' will silently ignore any extra attributes, and 'allow' will assign the attributes to the model.
allow_mutation
whether or not models are faux-immutable, i.e. whether __setattr__ is allowed (default: True)
* `use_enum_values`
whether to populate models with the value property of enums, rather than the raw enum. This may be useful if you want to serialise model.dict() later (default: False)
* `fields`
a dict containing schema information for each field; this is equivalent to using the Field class (default: None)
* `validate_assignment`
whether to perform validation on assignment to attributes (default: False)
* `allow_population_by_field_name`
whether an aliased field may be populated by its name as given by the model attribute, as well as the alias (default: False)
* `error_msg_templates`
a dict used to override the default error message templates. Pass in a dictionary with keys matching the error messages you want to override (default: {})
* `arbitrary_types_allowed`
whether to allow arbitrary user types for fields (they are validated simply by checking if the value is an instance of the type). If False, RuntimeError will be raised on model declaration (default: False). See an example in Field Types.
* `orm_mode`
whether to allow usage of ORM mode
* `getter_dict`
a custom class (which should inherit from GetterDict) to use when decomposing ORM classes for validation, for use with orm_mode
* `alias_generator`
a callable that takes a field name and returns an alias for it
* `keep_untouched`
a tuple of types (e.g. descriptors) for a model's default values that should not be changed during model creation and will not be included in the model schemas. Note: this means that attributes on the model with defaults of this type, not annotations of this type, will be left alone.
* `schema_extra`
a dict used to extend/update the generated JSON Schema, or a callable to post-process it; see Schema customization
* `json_loads`
a custom function for decoding JSON; see custom JSON (de)serialisation
* `json_dumps`
a custom function for encoding JSON; see custom JSON (de)serialisation
* `json_encoders`
a dict used to customise the way types are encoded to JSON; see JSON Serialisation

El que más destacamos acá es `arbitrary_types_allowed`, que nos permite validar cualquier tipo de datos (desde otras librerías).

In [12]:
from pydantic import BaseModel
from pandas import DataFrame

In [13]:
# error
class User(BaseModel):
    id: int
    df: DataFrame

RuntimeError: no validator found for <class 'pandas.core.frame.DataFrame'>, see `arbitrary_types_allowed` in Config

In [ ]:
# correcto
class User(BaseModel):
    id: int
    df: DataFrame
        
    class Config:
        arbitrary_types_allowed = True

In [14]:
# ejemplo      
User(
    id = 1,
    df = DataFrame({'a':[1,2],'b':[3,4]})
)

User(id=1, name='Jane Doe')